# Plotting example

Create example plots of the residual circulation with contours of mean isentropes, eddy-diffusivity with insignificance hatching, EP-flux divergence with EP-flux vectors, and the generalized TEM potential temperature budget. Note that this repository does not contain any meteorological data. ERA5 reanalysis data can be downloaded from the Copernicus Climate Data Store doi:10.24381/cds.bd0915c6.

In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as stats
import matplotlib.pyplot as plt
import cmocean 

from numba import float64, guvectorize
from generalized_TEM import Generalized, epQuiver

## Create bootstrap samples and estimate confidence intervals

In [ ]:
def pseudo_index(N,p=0.1):
    '''
        Pseudo time index to create stationary bootstrap sample following Politis, D. N. and Romano, J. P. (1994).
    '''
    index = []
    count = 1
    
    while len(index) < N:
        pos = int(stats.uniform.rvs(scale=N))
        length = stats.geom.rvs(p)
        index.extend(list(range(pos,pos+length)))

    index = np.mod(index,N)

    return index[:N]


@guvectorize(
    "(float64[:],float64[:],float64[:])",
    "(n), (m) -> (m)",
    forceobj=True
)    
def icdf(a,p,out):
    '''
        Inverse empirical cummulative distribution function of array at percentiles p
    '''
    sort = np.sort(a)
    out[:] = sort[np.int64(p*len(a))]


def confid(dist,alpha):
    '''
        Estimate confidence intervals using the inverse empirical cummulative distribution function 
        for distrubution of bootstrap samples.
    '''
    p = xr.DataArray([alpha/2, 1-alpha/2],dims=('percentile'))
    values = xr.apply_ufunc(icdf,
                          *(dist,p),
                          input_core_dims=[['random'],['percentile']],
                          output_core_dims=[['percentile']],
                          dask='parallelized',
                          output_dtypes=[[dist.dtype]])
    values['percentile'] = p
    
    return values


# Assess signficance by checking whether a confidence interval contains zero
significance = lambda confid: np.add(0 < confid.min('percentile'), 0 > confid.max('percentile'))

In [ ]:
# import dataset using xarray
data = xr.open_dataset('./variables_on_pressure_levels.nc',chunks=dict())

# compute generalized TEM (Greatbatch et al., 2022)
tem = Generalized(data['t'],('longitude','time'), H=7000)

# store bootstrap sample
N = len(data.time)
bootstrap = data.isel(time=pseudo_index(N))
diffusivity = tem.diffusivity(bootstrap['v'].sel(level=slice(50,1000)),
                              bootstrap['w'].sel(level=slice(50,1000)),
                              bootstrap['t'].sel(level=slice(50,1000)),
                              order=3)
xr.Dataset(dict(diff=diffusivity)).to_netcdf('./bootstrap_sample_0.nc')

## Plot the residual circulation with contours of mean isentropes

In [ ]:
# mean potential temperatures
mean = tem.theta.sel(level=slice(50,1000))

# streamfunctions
Chi = tem.streamfunction(data['v'].sel(level=slice(50,1000)))
eChi = tem.eChi(data['v'].sel(level=slice(50,1000)),
                data['w'].sel(level=slice(50,1000)),
                data['t'].sel(level=slice(50,1000)),
                order=3)

# plotting
ax = plt.axes()

C1 = (eChi + Chi).plot.contourf(ax=ax,levels=np.linspace(-6000,6000,33),x='latitude',
                                extend='both',cmap=cmocean.cm.balance,add_colorbar=False)
C2 = mean.plot.contour(ax=ax,levels=np.arange(260,560,20),colors='k',add_colorbar=False)
plt.clabel(C2)

# config axes
ax.set_xticks([-60,-30,0,30,60])
ax.set_ylabel('pressure [hPa]')
ax.set_xlabel('latitude [°N]')
ax.set_ylim(ax.get_ylim()[::-1])
ax.set_yscale('log')

# colorbar
cbar = plt.colorbar(C1,ax=ax,orientation='horizontal',pad=0.2,fraction=0.1,
                    ticks=[-6000,-4500,-3000,-1500,0,1500,3000,4500,6000],)
cbar.set_label(r'mass streamfunction [kg m$^{-1}$ s$^{-1}$]',weight='bold')
cbar.ax.set_xticklabels([-6000,-4500,-3000,-1500,0,1500,3000,4500,6000],rotation=45)

## Plot eddy-diffusivity with insignificance hatching

In [ ]:
# eddy-diffusivity 
diff = tem.diffusivity(data['v'].sel(level=slice(50,1000)),
                       data['w'].sel(level=slice(50,1000)),
                       data['t'].sel(level=slice(50,1000)),
                       order=3)

# significance
dist = xr.open_mfdataset('./bootstrap_sample_*.nc',combine='nested',concat_dim='random')['diff'].load()

sig = confid(dist,0.05)
sig = significance(sig)


# plotting
ax = plt.axes()

C1 = diff.plot.contourf(ax=ax,levels=np.linspace(-15,15,33),x='latitude',
                             extend='both',cmap=cmocean.cm.balance,add_colorbar=False)
sig.astype(np.double).plot.contourf(ax=ax,y='level',levels=[0,0.5,1],hatches=['\\',''],
                                         alpha=0,add_colorbar=False)

# config axes
ax.set_xticks([-60,-30,0,30,60])
ax.set_ylabel('pressure [hPa]')
ax.set_xlabel('latitude [°N]')
ax.set_ylim(ax.get_ylim()[::-1])
ax.set_yscale('log')

# colorbar
cbar = plt.colorbar(C1,ax=ax,orientation='horizontal',pad=0.2,fraction=0.1,
                    ticks=[-15,-11.25,-7.5,-3.75,0,3.75,7.5,11.25,15])
cbar.set_label(r'eddy diffusivity [m$^{2}$ s$^{-1}$]',weight='bold')
cbar.ax.set_xticklabels([-15,-11.25,-7.5,-3.75,0,3.75,7.5,11.25,15],rotation=45)

## Plot EP-flux divergence with EP-flux vectors

In [ ]:
# EP-flux
yflux, zflux = tem.EP_flux(data['u'].sel(level=slice(50,1000)),
                           data['v'].sel(level=slice(50,1000)),
                           data['w'].sel(level=slice(50,1000)),
                           data['t'].sel(level=slice(50,1000)),
                           order=3)
div = tem.spher_div(yflux,zflux)


# plotting
fig = plt.figure()
ax = plt.axes()

C1 = div.plot.contourf(ax=ax,levels=np.linspace(-600,600,33),x='latitude',
                       extend='both',cmap=cmocean.cm.balance,add_colorbar=False)

# config axes
ax.set_xticks([-60,-30,0,30,60])
ax.set_ylabel('pressure [hPa]')
ax.set_xlabel('latitude [°N]')
ax.set_ylim(ax.get_ylim()[::-1])
ax.set_yscale('log')

# colorbar
cbar = plt.colorbar(C1,ax=ax,orientation='horizontal',pad=0.2,fraction=0.1,
                    ticks=[-600,-450,-300,-150,0,150,300,450,600],)
cbar.set_label(r'EP-flux divergence [kg m$^{-1}$ s$^{-2}$]',weight='bold')
cbar.ax.set_xticklabels([-600,-450,-300,-150,0,150,300,450,600],rotation=45)

# Quiver 
epQuiver(tem,fig,ax,yflux,zflux)

## Plot the generalized TEM potential temperature budget

In [ ]:
def thermodynamics(tem,mean,eddy):
    
    # mean potential temperature gradient
    grad_y, grad_z = tem.grad(mean['mean'])
    
    # compute advection
    overturning = mean['Chi'] + eddy['eChi']
    vs, ws = tem.rot_grad((-1) * overturning)
    advection = (-1) * (vs*grad_y + ws*grad_z)
    advection = advection * 86400
    
    # compute diffusive flux convergence
    convergence = tem.spher_div(eddy['diff']*grad_y, eddy['diff']*grad_z)
    convergence = convergence * 86400
    
    # Plotting
    fig, axes = plt.subplots(nrows=1,ncols=3,sharex=True,sharey=True,figsize=(12,6))
    axes = axes.flatten()
    
    levels = np.linspace(-6,6,33)
    mean_levels = np.arange(260,560,20)
    
    advection.plot.contourf(ax=axes[0],levels=levels,x='latitude'
                            ,extend='both',cmap=cmocean.cm.balance,add_colorbar=False)
    convergence.plot.contourf(ax=axes[1],levels=levels,x='latitude'
                              ,extend='both',cmap=cmocean.cm.balance,add_colorbar=False)
    C = (-advection-convergence).plot.contourf(ax=axes[2],levels=levels,x='latitude'
                                              ,extend='both',cmap=cmocean.cm.balance,add_colorbar=False)
    
    ax = axes[0].twinx()
    ax.set_title('Advection',weight='bold')
    ax.set_yticks([])
    ax = axes[1].twinx()
    ax.set_title('Diffusion',weight='bold')
    ax.set_yticks([])
    ax = axes[2].twinx()
    ax.set_title(r'Steady-state $\overline{Q}$',weight='bold')
    ax.set_yticks([])
    
    for ax in axes:
        C1 = mean['mean'].plot.contour(ax=ax,levels=mean_levels,colors='k',add_colorbar=False)
        plt.clabel(C1)
        
        ax.set_ylabel(None)
        ax.set_xlabel('latitude [°N]')
        ax.set_xticks([-60,-30,0,30,60])
    
    axes[0].set_ylabel('pressure [hPa]')
    
    axes[0].set_ylim(ax.get_ylim()[::-1])
    axes[0].set_yscale('log')
    
    cbar = plt.colorbar(C,ax=axes,orientation='horizontal',pad=0.15,fraction=0.1,
                        label=r'heating [K day$^{-1}$]')
        
# mean flow diagnostics 
Chi = tem.streamfunction(data['v'].sel(level=slice(50,1000)))
mean = tem.theta.sel(level=slice(50,1000))
    
mean = xr.Dataset(dict(Chi=Chi,mean=mean))

# eddy diagnostics
eChi = tem.eChi(data['v'].sel(level=slice(50,1000)),
                data['w'].sel(level=slice(50,1000)),
                data['t'].sel(level=slice(50,1000)),
                order=3)
diff = tem.diffusivity(data['v'].sel(level=slice(50,1000)),
                       data['w'].sel(level=slice(50,1000)),
                       data['t'].sel(level=slice(50,1000)),
                       order=3)

eddy = xr.Dataset(dict(eChi=eChi,diff=diff))


thermodynamics(tem,mean,eddy)